# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».<br>
<br>
В распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта «comparison_on_tariffs»).<br><br>
**Задача:** <br>
Построение модель, которая выберет подходящий тариф клиенту тариф на основании данных о поведении клиентов
<br><br>
**План выполнения проекта:**<br>
1) Изучение общей информации о данных.<br>
2) Разделение данных на выборки.<br>
3) Исследование качества разных моделей.<br>
4) Проверка качества модели на тестовой выборке.<br>
5) Проверка модели на вменяемость.<br>
6) Вывод

## Откройте и изучите файл

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


**Описание данных:**<br>
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:<br>
•	сalls — количество звонков,<br>
•	minutes — суммарная длительность звонков в минутах,<br>
•	messages — количество sms-сообщений,<br>
•	mb_used — израсходованный интернет-трафик в Мб,<br>
•	is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [4]:
# сводная информация о таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
# проверяем наличие дубликоатов
df.duplicated().sum()

0

Явные дубликаты отсутствуют.

In [6]:
# проверяем пропущенные значения
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

Пропусков в данных нет.

In [7]:
# сменим тип данных столбца messages
df['messages'] = df['messages'].astype(int)
display(df.head())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83,19915.42,0
1,85.0,516.75,56,22696.96,0
2,77.0,467.66,86,21060.45,0
3,106.0,745.53,81,8437.39,1
4,66.0,418.74,1,14502.75,0


In [8]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [9]:
df.columns

Index(['calls', 'minutes', 'messages', 'mb_used', 'is_ultra'], dtype='object')

Вывод:<br> - в таблице 3214 строк - объекты, 5 столбцов - признаки, <br>-дубликаты отсутсвуют, <br> - пропуски в данных отсутствуют, <br> - тип данных столбца 'messages' ('количество sms-сообщений') заменен на int (целое число), тип данных остальных столбцов в порядке, <br> - названия столбцов не требуют изменений.

## Разбейте данные на выборки

Выделяем признаки и целевой признак

In [11]:
# признаки
features = df.drop(['is_ultra'], axis=1)

In [12]:
# целевой признак
target = df['is_ultra']

Обучающий (тренировочный) набор данных  (выборка) - набор данных, на котором будет обучаться алгоритм машинного обучения<br>
Тестовая выборка необходима для оценки работы готовой модели, набор данных, на котором будет проверяться качество обученной модели. Что бы точно знать, что модель не вызубрила ответы.<br>
Валидационная выборка - набор данных, извлекаемый из исходного датасета, на которой проверяется качество работы алгоритма по время обучения модели. При валидации модели выбирают набор гиперпараметров, который соответствует лучшей модели.<br>
Спрятанной выборки нет, значит от основной для валидационной отделяют 20%, и 20% для тестовой, соотношение получается 3/1/1
(обучающая 60%, валидационная 20%, тестовая 20%).

In [13]:
# разделяем данные, выделяем обучающую и валидационную+тестовую (60/40)

df_train, df_train_testing = train_test_split(df, test_size=0.4, random_state=12345)

In [14]:
# разделяем df_train_testing данные, выделяем валидационную и тестовую (50/50)

df_test, df_valid = train_test_split(df_train_testing, test_size=0.5, random_state=12345)

df_train - обучающая выборка<br>
df_train_testing - тестовая + валидационная выборки<br>
df_test - тестовая выборка<br>
df_valid - валидационная выборка

Создаем переменные для признаков и для целевого признака для обучающей, тестовой и валидационной выборки

In [15]:
# для обучающей
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

In [16]:
# для тестовой
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

In [17]:
# для валидационной
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

In [18]:
# размеры полученных выборок
print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)
print(features_valid.shape)
print(target_valid.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)
(643, 4)
(643,)


## Исследуйте модели

### Логистическая регрессия

In [19]:
model_log = LogisticRegression(random_state=12345, solver='lbfgs')
model_log.fit(features_train, target_train)
prediction_valid_log = model_log.predict(features_valid)
print('Accuracy LogisticRegression:', accuracy_score(target_valid, prediction_valid_log))

Accuracy LogisticRegression: 0.6842923794712286


### Дерево решений

In [20]:
best_accuracy = 0
best_depth = 0
for depth in range(1, 15):
    model_dr = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_dr.fit(features_train, target_train)
    prediction_valid_dr = model_dr.predict(features_valid)
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, prediction_valid_dr))
    if accuracy_score(target_valid, prediction_valid_dr) > best_accuracy:
        best_accuracy = accuracy_score(target_valid, prediction_valid_dr)
        best_depth = depth
print('Наилучшее качество')
print('Accuracy DecisionTreeClassifier:', best_accuracy, 'Глубина дерева:', best_depth)

max_depth = 1 : 0.7356143079315708
max_depth = 2 : 0.7744945567651633
max_depth = 3 : 0.7791601866251944
max_depth = 4 : 0.7744945567651633
max_depth = 5 : 0.7838258164852255
max_depth = 6 : 0.776049766718507
max_depth = 7 : 0.7993779160186625
max_depth = 8 : 0.7931570762052877
max_depth = 9 : 0.7807153965785381
max_depth = 10 : 0.7884914463452566
max_depth = 11 : 0.7744945567651633
max_depth = 12 : 0.7807153965785381
max_depth = 13 : 0.7713841368584758
max_depth = 14 : 0.76049766718507
Наилучшее качество
Accuracy DecisionTreeClassifier: 0.7993779160186625 Глубина дерева: 7


### Случайный лес

In [21]:
best_model = None
best_accuracy = 0
best_est = 0
best_depth = 0
for est in range(5, 50, 5):
    for depth in range(1, 15):
        model_sl = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_sl.fit(features_train, target_train)
        prediction_valid_sl = model_sl.predict(features_valid)
        if accuracy_score(target_valid, prediction_valid_sl) > best_accuracy:
            best_model = model_sl
            best_accuracy = accuracy_score(target_valid, prediction_valid_sl)
            best_est = est
            best_depth = depth
print('Наилучшее качество')
print('Accuracy RandomForestClassifier:', 
      best_accuracy, 'Количество деревьев:', est, 'Глубина дерева:', best_depth)

Наилучшее качество
Accuracy RandomForestClassifier: 0.8133748055987559 Количество деревьев: 45 Глубина дерева: 9


Лучший показатель точности (accuracy - "доля правильных ответов") - Случайный лес: качество: 0.8133748055987559, количество деревьев: 45, глубина дерева: 9.

## Проверьте модель на тестовой выборке

In [22]:
model_log = LogisticRegression(random_state=12345, solver='lbfgs')
model_log.fit(features_train, target_train)
prediction_test_log = model_log.predict(features_test)
print('Accuracy LogisticRegression:', accuracy_score(target_test, prediction_test_log))

Accuracy LogisticRegression: 0.7107309486780715


In [23]:
model_dr = DecisionTreeClassifier(random_state=12345, max_depth=7)
model_dr.fit(features_train, target_train)
prediction_test_dr = model_dr.predict(features_test)
print('Accuracy DecisionTreeClassifier:', accuracy_score(target_test, prediction_test_dr))

Accuracy DecisionTreeClassifier: 0.7822706065318819


In [24]:
model_sl = RandomForestClassifier(random_state=12345, n_estimators=45, max_depth=9)
model_sl.fit(features_train, target_train)
prediction_test_sl = model_sl.predict(features_test)
print('Accuracy RandomForestClassifier:', accuracy_score(target_test, prediction_test_sl))

Accuracy RandomForestClassifier: 0.7947122861586314


Результаты проверки на тестовой модели: <br>
- Логистическая регрессия: Accuracy: 0.7107309486780715 <br>
- Дерево решений: Accuracy: 0.7822706065318819 <br>
- Случайный лес: Accuracy: 0.7947122861586314 <br>
Тестовое качество расположилось в такой же последовательности. Лучшший результат - случайный лес - Accuracy: 0.7947122861586314 (Показатель на 2% ниже, чем в валидационной выборке)

## (бонус) Проверьте модели на адекватность

Внедрение модели должно быть оправдано. Необходимо сравнивать модель со случайной, так мы можем оценить ее адекватность (или проверить на вменяемость, sanity check).

DummyClassifier-это классификатор,который делает прогнозы,используя простые правила. Этот классификатор полезен в качестве простой основы для сравнения с другими (реальными) классификаторами. 

In [25]:
# Стратегию  использования для создания прогнозов возьмем «most_frequent»: 
# всегда предсказывает наиболее часто встречающуюся метку в обучающем наборе, random_state прежний, 12345.

model_dummyclassifier = DummyClassifier(strategy='most_frequent', random_state=12345)
model_dummyclassifier.fit(features_train, target_train)
prediction_dummyclassifier = model_dummyclassifier.predict(features_test)
print('Accuracy DummyClassifier:', accuracy_score(target_test, prediction_dummyclassifier))
print('Accuracy RandomForestClassifier:', accuracy_score(target_test, prediction_test_sl))

Accuracy DummyClassifier: 0.7060653188180405
Accuracy RandomForestClassifier: 0.7947122861586314


In [26]:
# Стратегию  использования для создания прогнозов возьмем «uniform»: 
# «однородный»: генерирует прогнозы случайным образом, random_state прежний, 12345.

model_dummyclassifier = DummyClassifier(strategy='uniform', random_state=12345)
model_dummyclassifier.fit(features_train, target_train)
prediction_dummyclassifier = model_dummyclassifier.predict(features_test)
print('Accuracy DummyClassifier:', accuracy_score(target_test, prediction_dummyclassifier))
print('Accuracy RandomForestClassifier:', accuracy_score(target_test, prediction_test_sl))

Accuracy DummyClassifier: 0.5038880248833593
Accuracy RandomForestClassifier: 0.7947122861586314


Подробное вычисление случайных ответов. Предположим это модель, которая предсказывает все объекты случайным образом. С вероятностью 50/50 ответ будет либо "0", либо "1".<br> Accuracy = 0.5 * (доля угаданных 1) + 0,5 * (доля угаданных 0).<br> Ответы модели не связаны с правильными ответами, поэтому вероятность угадать 1 равна 50 % — так же и для 0. Accuracy будет равна 0.5.

## Вывод

<br><br> Исходные данные: таблица из 3214 строк (объекты) и 5 столбцов (признаки), дубликаты отсутсвуют, пропуски в данных отсутствуют, тип данных столбца 'messages' ('количество sms-сообщений') заменен на int (целое число), тип данных остальных столбцов в порядке, названия столбцов не требуют изменений.<br><br> Данные разбиты на выборки: df_train - обучающая выборка, df_test - тестовая выборка
df_valid - валидационная выборка, в соотношении 3:1:1 (60%, 20% и 20% соответственно).<br><br> Были исследованы следующие модели: Логистическая регрессия (LogisticRegression), Дерево решений (DecisionTreeClassifier) и Случайный лес (RandomForestClassifier).<br><br> **Лучший показатель точности** (accuracy - "доля правильных ответов") - Случайный лес: Accuracy RandomForestClassifier: 0.8133748055987559, количество деревьев: 45, глубина дерева: 9 (Проверка на тестовой выборке: Accuracy RandomForestClassifier: 0.8133748055987559 Количество деревьев: 45 Глубина дерева: 9.)<br><br> В оставшихся моделях получены следующие значения:<br> - Дерево решений- Accuracy DecisionTreeClassifier: 0.7993779160186625 Глубина дерева: 7 (Проверка на тестовой выборке: Accuracy DecisionTreeClassifier: 0.7822706065318819)<br> - Логистическая регрессия- Accuracy LogisticRegression: 0.6842923794712286 (Проверка на тестовой выборке: Accuracy LogisticRegression: 0.7107309486780715<br><br> При оценке лучшей модели на адекватность (или проверить на вменяемость)  Accuracy получили ниже, чем в нашей модели (при стратегии  «most_frequent» (всегда предсказывает наиболее часто встречающуюся метку в обучающем наборе)Accuracy DummyClassifier: 0.7060653188180405; при стратегии «uniform» (генерирует прогнозы случайным образом) Accuracy DummyClassifier: 0.5038880248833593). Следовательно, модель RandomForestClassifier адекватна и эффективна, и подходит для задачи классификации, которая выберет подходящий тариф.